<a href="https://colab.research.google.com/github/ymoslem/PyTorchNLP/blob/main/Ex1-TextClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Classification with PyTorch
Source: https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html

# Load the dataset

In [130]:
import torch
from torchtext.datasets import AG_NEWS
train_iter = AG_NEWS(split="train")

In [131]:
type(train_iter)

torchtext.data.datasets_utils._RawTextIterableDataset

In [132]:
next(train_iter)

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

In [133]:
len(train_iter)

120000

# Preprocessing data and building vocabulary

In [134]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

# Create a tokenizer object
tokenizer = get_tokenizer("basic_english")
# Get the training split from AG_NEWS dataset
train_iter = AG_NEWS(split="train")

# Yield an iterator of tokenized text lines
def yield_tokens(data_iter):
  for class_id, text_line in data_iter:
    yield tokenizer(text_line)

# Build Vocabulary
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
# Value of default index. This index will be returned when OOV token is queried.
vocab.set_default_index(vocab["<unk>"])

In [135]:
# Test vocab example
print(vocab(['here', 'is', 'an', 'example']))  # get IDs for a list of tokens
print(vocab["test"])  # a word in vocab
print(vocab["<unk>"]) # <unk> special token
print(vocab["adf4"])  # any oov token

[475, 21, 30, 5297]
289
0
0


# Generate Data Batches

In [136]:
import torch
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
  label_list, text_list, offsets = [], [], [0]
  for (class_id, text_line) in batch:
    class_id = int(class_id) - 1
    label_list.append(class_id)
    # Tokenize the text line
    text_line_tokens = tokenizer(text_line)
    # Convert tokens to ID based on vocab 
    text_line_ids = vocab(text_line_tokens)
    # Convert text_line_ids to a PyTorch tensor
    text_line_tensor = torch.tensor(text_line_ids, dtype=torch.int64)
    # Add to text_list
    text_list.append(text_line_tensor)
    # Offsets
    offsets.append(text_line_tensor.size(0))
  
  # Convert label_list to a PyTorch tensor
  label_list = torch.tensor(label_list, dtype=torch.int64)
  # Create offsets. An offset is a tensor of delimiters to represent the beginning index of the individual sequence in the text tensor.
  offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
  # Concatenate the given sequence of seq tensors in the given dimension
  text_list = torch.cat(text_list)

  return label_list.to(device), text_list.to(device), offsets.to(device)


# Create a dataset iterator
train_iter = AG_NEWS(split="train")
# Create a DataLoader object
dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

# Define the model

In [137]:
from torch import nn

class TextClassificationModel(nn.Module):
  def __init__(self, vocab_size, embed_dim, num_class):
    super(TextClassificationModel, self).__init__()
    # nn.EmbeddingBag module requires no padding here since the text lengths are saved in offsets
    self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
    self.fc = nn.Linear(embed_dim, num_class)
    self.init_weights()

  def init_weights(self):
    initrange = 0.5
    self.embedding.weight.data.uniform_(-initrange, initrange)
    self.fc.weight.data.uniform_(-initrange, initrange)
    self.fc.bias.data.zero_()

  def forward(self, text, offsets):
    embedded = self.embedding(text, offsets)
    return self.fc(embedded)

In [138]:
train_iter = AG_NEWS(split="train")
num_class = len(set([line[0] for line in train_iter]))
vocab_size = len(vocab)
embed_dim = 64
model = TextClassificationModel(vocab_size, embed_dim, num_class).to(device)

print("Number of classes:", num_class)
print("Vocab size:", vocab_size)

Number of classes: 4
Vocab size: 95811


In [139]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset

epochs = 10
lr = 5
batch_size = 64

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=1.0, gamma=0.1)
total_acc = None
train_iter, test_iter = AG_NEWS()
# Convert iterable-style dataset to map-style dataset
main_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(main_dataset) * 0.95)
num_valid = len(main_dataset) - num_train
train_dataset, valid_dataset = random_split(main_dataset, [num_train, num_valid])

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_batch)

In [140]:
train_dataset[0]

(3,
 'Kidde rejects United Technologies bid United Technologies made a 1.35 billion pound bid for Kidde, but the fire-fighting equipment maker has rejected the offer, saying it undervalues the company.')

# Create the training and evaluation functions

In [141]:
import time

def train(dataloader):
  model.train()
  total_acc, total_count = 0, 0
  log_interval = 500
  start_time = time.time()

  for idx, (label, text, offsets) in enumerate(dataloader):
    optimizer.zero_grad()
    predicted_label = model(text, offsets)
    loss = criterion(predicted_label, label)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.1)
    optimizer.step()
    total_acc += (predicted_label.argmax(1) == label).sum().item()
    total_count += label.size(0)
    if idx % log_interval == 0 and idx > 0:
      elapsed = time.time() - start_time
      print('| epoch {:3d} | {:5d}/{:5d} batches | accuracy {:8.3f}'.format(epoch, idx, len(dataloader), total_acc/total_count))
      total_acc, total_count = 0, 0
      start_time = time.time()

def evaluate(dataloader):
  model.eval()
  total_acc, total_count = 0, 0

  with torch.no_grad():
    for idx, (label, text, offsets) in enumerate(dataloader):
      predicted_label = model(text, offsets)
      loss = criterion(predicted_label, label)
      total_acc += (predicted_label.argmax(1) == label).sum().item()
      total_count += label.size(0)
    return total_acc/total_count 

# Run the training loop

In [142]:
# Training loop
for epoch in range(1, epochs+1):
  epoch_start_time = time.time()
  train(train_dataloader)
  acc_val = evaluate(valid_dataloader)
  if total_acc is not None and total_acc > acc_val:
    scheduler.step()
  else:
    total_acc = acc_val
  print("-" * 59)
  print('| end of epoch {:3d} | time: {:5.2f}s | valid accuracy {:8.3f} '.format(epoch, time.time() - epoch_start_time, acc_val))
  print('-' * 59)


| epoch   1 |   500/ 1782 batches | accuracy    0.680
| epoch   1 |  1000/ 1782 batches | accuracy    0.855
| epoch   1 |  1500/ 1782 batches | accuracy    0.875
-----------------------------------------------------------
| end of epoch   1 | time:  6.90s | valid accuracy    0.884 
-----------------------------------------------------------
| epoch   2 |   500/ 1782 batches | accuracy    0.894
| epoch   2 |  1000/ 1782 batches | accuracy    0.900
| epoch   2 |  1500/ 1782 batches | accuracy    0.903
-----------------------------------------------------------
| end of epoch   2 | time:  6.90s | valid accuracy    0.901 
-----------------------------------------------------------
| epoch   3 |   500/ 1782 batches | accuracy    0.911
| epoch   3 |  1000/ 1782 batches | accuracy    0.915
| epoch   3 |  1500/ 1782 batches | accuracy    0.914
-----------------------------------------------------------
| end of epoch   3 | time:  6.86s | valid accuracy    0.894 
-------------------------------

# Evaluate the model on the test dataset

In [144]:
print('Checking the results of test dataset.')
acc_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(acc_test))

Checking the results of test dataset.
test accuracy    0.906


# Evaluate the model on out-of-domain data

In [154]:
ag_news_label = {
    1: "World",
    2: "Sports",
    3: "Business",
    4: "Sci/Tec"
}

def predict(text):
  with torch.no_grad():
    # Tokenize the text
    text_tokens = tokenizer(text)
    # Convert tokens to ID based on vocab 
    text_ids = vocab(text_tokens)
    # Convert text_ids to a PyTorch tensor
    text = torch.tensor(text_ids, dtype=torch.int64)
    output = model(text, torch.tensor([0]))
    return output.argmax(1).item() + 1

text = "MEMPHIS, Tenn. – Four days ago, Jon Rahm was \
    enduring the season’s worst weather conditions on Sunday at The \
    Open on his way to a closing 75 at Royal Portrush, which \
    considering the wind and the rain was a respectable showing. \
    Thursday’s first round at the WGC-FedEx St. Jude Invitational \
    was another story. With temperatures in the mid-80s and hardly any \
    wind, the Spaniard was 13 strokes better in a flawless round. \
    Thanks to his best putting performance on the PGA Tour, Rahm \
    finished with an 8-under 62 for a three-stroke lead, which \
    was even more impressive considering he’d never played the \
    front nine at TPC Southwind."

model = model.to("cpu")
prediction = predict(text)
print("This is a %s news" %ag_news_label[prediction])


This is a Sports news
